<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Final project - Introduction to DS
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Data preprocessing & Basic exploration
</div>

## Import

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import isodate

## Read raw data from file

In [2]:
raw_df = pd.read_csv("../data/raw/video_data_raw.csv")
raw_df.sample(n=5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
43779,b_wsqYm6p4w,itversity,Getting Started - Setup VMWare Fusion,Connect with me or follow me at\r\nhttps://www...,"['Apache Hadoop', 'Apache Spark', 'Big Data En...",2016-07-31T17:28:08Z,849.0,3.0,NaN,1.0,PT3M35S,hd,False
59639,rIVb1p69Nuw,Art of Visualization,Creating your first function,Get the Full course here: https://www.udemy.co...,"['R programming', 'R programming tutorial', 'R...",2016-08-22T18:50:35Z,794.0,3.0,NaN,0.0,PT8M48S,hd,False
2041,-Inc1be4zSg,Alex The Analyst,The Hiring Process from an Interviewer's Persp...,The hiring process can be long and extremely d...,"['Data Analyst', 'How to become a data analyst...",2020-09-03T15:38:28Z,19247.0,666.0,NaN,83.0,PT27M4S,hd,False
41726,wP7BhXrJKR8,itversity,Setup Ubuntu VM on GCP - Setup Docker,This video is all about Setup Ubuntu VM on GCP...,"['Setup Ubuntu VM on GCP', 'Setup Ubuntu VM', ...",2020-11-28T16:31:14Z,879.0,5.0,NaN,0.0,PT8M17S,hd,False
8127,_26BQrhNWgc,Unfold Data Science,Dockers vs Virtual Machines | Dockers vs Conta...,"Hello ,\r\nMy name is Aman and I am a Data Sci...","['Dockers vs Virtual Machines', 'Dockers vs Co...",2022-09-07T14:29:32Z,1443.0,82.0,NaN,14.0,PT13M46S,hd,False


### How many rows and how many columns does the raw data have?

In [3]:
data_video_shape = raw_df.shape
print(f"Video data current shape: {data_video_shape}")

Video data current shape: (60032, 13)


Answer: Our raw video data has `60032 rows` and `13 columns`, which meets the given requirement: "Your dataset must have at least `5 fields` and `1000 observations`".

### What does each line mean? Does it matter if the lines have different meanings?

Answer: 
- Based on the observations of the dataset, we suppose that each line represents a unique set of details regarding individual YouTube videos. 
- Looking through the data table of the csv file, it seems there are no "out of place" lines.

### Does the raw data have duplicate rows?

In [4]:
# retrieve the index
index = raw_df.index
# create a Pandas Series indicating whether each index is duplicated or not
deDupSeries = index.duplicated(keep='first')
# calculate the number of duplicated rows
num_duplicated_rows = deDupSeries.sum()

In [5]:
if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated line!")
else:
    ext = "lines" if num_duplicated_rows > 1 else "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext)

Raw data have no duplicated line!


### What does each column mean?

<table style="width: 100%; border-collapse: collapse;">
  <thead>
    <tr>
      <th style="min-width: 10px; width: 30px; background-color: #04B1CC; color: white; font-size :15px; font-weight: bold; text-align: left; padding: 8px;">No</th>
      <th style="background-color: #04B1CC; color: white; font-size :15px; font-weight: bold; text-align: left; padding: 8px;">Columns</th>
      <th style="background-color: #04B1CC; color: white; font-size :15px; font-weight: bold; text-align: left; padding: 8px;">Meaning</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="font-size: 14px; text-align: left;">1</td>
      <td style="font-size: 14px; text-align: left;">video_id</td>
      <td style="font-size: 14px; text-align: left;">Unique identifier for the YouTube video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">2</td>
      <td style="font-size: 14px; text-align: left;">channelTitle</td>
      <td style="font-size: 14px; text-align: left;">The name of the channel that uploaded the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">3</td>
      <td style="font-size: 14px; text-align: left;">title</td>
      <td style="font-size: 14px; text-align: left;">Title of the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">4</td>
      <td style="font-size: 14px; text-align: left;">description</td>
      <td style="font-size: 14px; text-align: left;">Description or summary of the video content.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">5</td>
      <td style="font-size: 14px; text-align: left;">tags</td>
      <td style="font-size: 14px; text-align: left;">Keywords or tags associated with the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">6</td>
      <td style="font-size: 14px; text-align: left;">publishedAt</td>
      <td style="font-size: 14px; text-align: left;">Date and time when the video was published.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">7</td>
      <td style="font-size: 14px; text-align: left;">viewCount</td>
      <td style="font-size: 14px; text-align: left;">Number of views the video has accumulated.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">8</td>
      <td style="font-size: 14px; text-align: left;">likeCount</td>
      <td style="font-size: 14px; text-align: left;">Count of likes received by the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">9</td>
      <td style="font-size: 14px; text-align: left;">favoriteCount</td>
      <td style="font-size: 14px; text-align: left;">Deprecated; used to track how many times viewers added the video to their favorites.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">10</td>
      <td style="font-size: 14px; text-align: left;">commentCount</td>
      <td style="font-size: 14px; text-align: left;">Number of comments posted on the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">11</td>
      <td style="font-size: 14px; text-align: left;">duration</td>
      <td style="font-size: 14px; text-align: left;">Length of the video.</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">12</td>
      <td style="font-size: 14px; text-align: left;">definition</td>
      <td style="font-size: 14px; text-align: left;">Video resolution or quality (e.g., HD, SD).</td>
    </tr>
    <tr>
      <td style="font-size: 14px; text-align: left;">13</td>
      <td style="font-size: 14px; text-align: left;">caption</td>
      <td style="font-size: 14px; text-align: left;"> Indicates whether closed captions are available for the video.</td>
    </tr>
  </tbody>
</table>

❓ TODO: Liệu có thông tin nào chỉ đúng ở thời điểm dữ liệu được thu thập, sau đó nó có thể bị thay đổi
vd: count sẽ tăng lên/giảm xuống (lượt like); 

### What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?

In [6]:
raw_df.dtypes

video_id           object
channelTitle       object
title              object
description        object
tags               object
publishedAt        object
viewCount         float64
likeCount         float64
favouriteCount    float64
commentCount      float64
duration           object
definition         object
caption              bool
dtype: object

We notice that the columns `publishedAt` and `duration` are currently of object type. Given that they represent time periods, it is advisable to convert `publishedAt`  to datetime type. As the `duration` is in **ISO 8601** format, we'll convert it into an `integer` representing the total number of seconds

In [7]:
# convert publishedAt to datetime
raw_df["publishedAt"] = pd.to_datetime(raw_df["publishedAt"])
# convert duration to integer
raw_df['duration'] = raw_df['duration'].apply(lambda x: isodate.parse_duration(x))
raw_df['duration'] = raw_df['duration'].dt.total_seconds()
raw_df['duration'] = raw_df['duration'].astype(np.int64)

In [8]:
# TEST
raw_df.dtypes

video_id                       object
channelTitle                   object
title                          object
description                    object
tags                           object
publishedAt       datetime64[ns, UTC]
viewCount                     float64
likeCount                     float64
favouriteCount                float64
commentCount                  float64
duration                        int64
definition                     object
caption                          bool
dtype: object

### With each numerical column, how are values distributed?

📚 For columns with numeric data types, we will calculate some basic statistics:
- Percentage (from 0 to 100) of missing values
- The min
- The lower quartile (phân vị 25)
- The median (phân vị 50)
- The upper quartile (phân vị 75)
- The max

Then we will observe and comment to see if the results are unusual?

In [9]:
# YOUR CODE HERE
num_col_info_df = raw_df.select_dtypes(exclude=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100)

def median(df):
    return (df.quantile(0.5))

def lower_quartile(df):
    return (df.quantile(0.25))

def upper_quartile(df):
    return (df.quantile(0.75))

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration
missing_ratio,0.0,6.663113e-03,0.404784,100.0,1.279318,0.0
min,2006-10-25 10:28:09+00:00,0.000000e+00,0.000000,NaN,0.000000,0.0
lower_quartile,2019-02-18 04:02:18+00:00,6.220000e+02,11.000000,NaN,0.000000,293.0
median,2020-12-22 09:26:05.500000+00:00,3.059500e+03,55.000000,NaN,5.000000,695.0
upper_quartile,2022-07-05 06:04:02.500000+00:00,1.602175e+04,334.000000,NaN,27.000000,1786.0
max,2023-11-24 13:04:35+00:00,3.447645e+07,571358.000000,NaN,60054.000000,92218.0


Observation: We notice that some columns have missing data and need to be resolved before moving onto the next analysis steps. Other than that, the data does not appear to be unusual.

- We notice that `favouriteCount` does not have any value, so we can drop this feature

In [10]:
# drop favouriteCount
if "favouriteCount" in raw_df.columns:
    raw_df = raw_df.drop('favouriteCount', axis=1)

-  `viewCount`, `likeCount` and `commentCount` have little missing value, we'll fill them with their `median` and then convert them to integer

In [11]:
# fill missing value with median
missing_cols = ['viewCount', 'likeCount', 'commentCount']
for col in missing_cols:
    raw_df[col] = raw_df[col].fillna(raw_df[col].median().__round__(0))
    raw_df[col] = raw_df[col].astype(np.int64)

In [12]:
# TEST
raw_df.select_dtypes(exclude=['object', 'bool'])\
    .agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

,publishedAt,viewCount,likeCount,commentCount,duration
missing_ratio,0.0,0.00,0.0,0.0,0.0
min,2006-10-25 10:28:09+00:00,0.00,0.0,0.0,0.0
lower_quartile,2019-02-18 04:02:18+00:00,622.00,11.0,1.0,293.0
median,2020-12-22 09:26:05.500000+00:00,3060.00,55.0,5.0,695.0
upper_quartile,2022-07-05 06:04:02.500000+00:00,16017.25,332.0,26.0,1786.0
max,2023-11-24 13:04:35+00:00,34476453.00,571358.0,60054.0,92218.0


### With each categorical column, how are values distributed?

📚 For columns with non-numeric data types, we calculate:
- Percentage (from 0 to 100) of missing values
- Number of different values (and we do not consider missing values)
- Show a few values and percentage (from 0 to 100) of each value sorted by decreasing percentage (we do not consider missing values, the ratio is the ratio compared to the number of non-missing values)

Then we will observe and comment to see if the results are unusual?

In [13]:
pd.set_option('display.max_colwidth', 100) # For clearly
pd.set_option('display.max_columns', None) # For clearly

In [14]:
cat_col_info_df = raw_df.select_dtypes(include=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100)

def num_values(s):
    s = s.astype('str').str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.astype('str').str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,video_id,channelTitle,title,description,tags,definition,caption
missing_ratio,0.0,0.0,0.0,2.803505,18.956557,0.0,0.0
num_values,60032,160,59644,53351,35545,2,2
value_ratios,"{'WkqM0ndr42c': 0.0, 'ls8hEDjhKU0': 0.0, 'Fh0ArGT2_b0': 0.0, '_tHCoU5TZZg': 0.0, 'fXAvXLOUjlU': ...","{'itversity': 5.8, 'Databricks': 4.7, 'Great Learning': 3.1, 'Analytics India Magazine': 3.0, 'K...","{'TL': 0.1, 'Data Analyst MENTORSHIP - Q&A (while I drink coffee)': 0.1, 'Machine Learning Summ...","{'nan': 2.7, 'Connect with me or follow me at https://www.linkedin.com/in/durga0gadiraju https...","{'nan': 18.9, '['Databricks']': 2.2, '['CBMM', 'Center for Brains Minds and Machines', 'Artifici...","{'hd': 95.5, 'sd': 4.5}","{'False': 88.6, 'True': 11.4}"


Observation: We notice that some columns have missing data and need to be resolved before moving onto the next analysis steps. Other than that, the data does not appear to be unusual.

- In here, we will change the format of `tags` string.

In [15]:
raw_df['tags'] = raw_df['tags'].apply(lambda x: '|'.join(x.strip("[]").replace("'", "").split(', ')) if isinstance(x, str) else x)
raw_df['tags']

0                                                          python|data science|machine learning|scikit-learn
1                                                                   python|pandas|data analysis|data science
2                                                          python|machine learning|scikit-learn|data science
3                                                          python|machine learning|scikit-learn|data science
4                                                          python|machine learning|scikit-learn|data science
                                                        ...                                                 
60027    free fme license|fme license key|fme home license|fme software|fme|safe software fme|fme data in...
60028    s3connector|api to s3 fme|httpcaller|fme data integration|fme software|safe software|api to s3|f...
60029    temppathnamecreater|FME Transformer Guide|temporay paths fme|fme software|fme software tutorial|...
60030    csv to sha

### Save the processed data

In [16]:
print(f"Total number of features: {raw_df.shape[1]}")
raw_df.dtypes

Total number of features: 12


video_id                     object
channelTitle                 object
title                        object
description                  object
tags                         object
publishedAt     datetime64[ns, UTC]
viewCount                     int64
likeCount                     int64
commentCount                  int64
duration                      int64
definition                   object
caption                        bool
dtype: object

In [17]:
# Save processed data to disk
raw_df.to_csv("../data/processed/" + "video_data_processed.csv", index=False)